# 🤒 Epidemic mitigation project (Riccardo Brioschi, Francesca Venturi)

This notebook contains the execution code of the *epidemic mitigation process* carried out by Riccardo Brioschi and Francesca Venturi. 

Moreover, not only does it contain the code, it also includes comments and discussions about results, coherently with the requirements of the project.

## Importing useful packages and Initializing the model class

In [ ]:
# Importing useful library
import matplotlib.pyplot as plt
from gym import spaces

import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

"""Environment imports"""
from epidemic_env.env       import Env, Log
from epidemic_env.dynamics  import ModelDynamics, Observation
from epidemic_env.visualize import Visualize
from epidemic_env.agent     import Agent

"""Pytorch and numpy imports"""
import numpy as np
import torch

%matplotlib inline

from os import makedirs
from shutil import rmtree

seed = 1

import warnings
warnings.filterwarnings('ignore')

In [ ]:
dyn = ModelDynamics('config/switzerland.yaml')   # load the switzerland map

## Question 1

We first define the possible actions, encoding them as integers. 

At this stage of the problem, we can identify 5 different possible actions, as we do not take into account the possibily to combine different decisions.

In [ ]:
ACTION_NULL = 0
ACTION_CONFINE = 1
ACTION_ISOLATE = 2
ACTION_HOSPITAL = 3
ACTION_VACCINATE = 4


def action_preprocessor(a:torch.Tensor, dyn:ModelDynamics):
    action = { # DO NOTHING
        'confinement': False, 
        'isolation': False, 
        'hospital': False, 
        'vaccinate': False,
    }
    
    if a == ACTION_CONFINE:
        action['confinement'] = True
    elif a == ACTION_ISOLATE:
        action['isolation'] = True
    elif a == ACTION_VACCINATE:
        action['vaccinate'] = True
    elif a == ACTION_HOSPITAL:
        action['hospital'] = True
        
    return action

### Initializing the environment needed to answer question in part 1. 

To be coherent with the structure of the tutorial, we decide to encode the actions as integer numbers (that is what is required from the `action_space`) and to exploit the `action_preprocessor` to convert them as a dictionary. Notice that at this stage of the mini-project this is not strictly required, as everything should work equivalently without this change.

On the contrary, giving the `observation_reprocessor` as an input argument is essential to later plot and visualize the results.

In [ ]:
env = Env(  dyn, # We pass the dynamical model to the environment 
            action_space=None, # Here one could pass an openai gym action space that can then be sampled
            observation_space=None, # Here one could pass an openai gym obs space that can then be sampled
            action_preprocessor=action_preprocessor
            )

Since the action we take is always the same (actually, we do not take any action, which means we select `ACTION_NULL`), we initialize it accordingly to the `action_space` and we use it when simulating the epidemic dynamics in our environment.

In [ ]:
action = ACTION_NULL

In [ ]:
log = []
finished = False
obs, info = env.reset(seed) # here it is not seeded
for t in range(30):
    obs, R, finished, info = env.step(action) # always same actions
    log.append(info) # save the information dict for logging
    if finished:
        break

""" Parse the logs """
total = {p:np.array([getattr(l.total,p) for l in log]) for p in dyn.parameters}
cities = {c:{p:np.array([getattr(l.city[c],p) for l in log]) for p in dyn.parameters} for c in dyn.cities}
actions = {a:np.array([l.action[a] for l in log]) for a in log[0].action.keys()}

#### Plots

In [ ]:
[plt.plot(y) for y in total.values()]
plt.legend(total.keys())
plt.ylabel('number of people in each state')
plt.xlabel('time (in weeks)')
plt.title('Unmitigated Epidemic')
plt.show()

In [ ]:
plt.plot(total['infected'], label = 'infected')
plt.plot(total['dead'], label = 'dead')
plt.legend()
plt.ylabel('number of people in each state')
plt.xlabel('time (in weeks)')
plt.title('Unmitigated Epidemic')
plt.show()

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(10,10))
row = col = 0
for idx, city in enumerate(cities.keys()):
    
    row = idx // 3
    col = idx % 3
    ax[row,col].plot(cities[city]['infected'],label = 'infected')
    ax[row,col].plot(cities[city]['dead'],label = 'dead')
    ax[row,col].set_title(city)
    
# choose title + common legend
plt.tight_layout()

### DISCUSSION

## Question 2

### Question 2.a
We implement Pr. Russo’s Policy as a python class (subclass the Agent abstract class provided with the project files) and initialize the agent accordingly.

Pr. Russo's Policy consists in confining (`ACTION_CONFINE`) the popoluation for 4 weeks once the infected amount of people exceeds 20000 units. The *confinement action* is not debatable during the confinment. This means that the confinments happen in blocks of (at least) 4 weeks each.

In [ ]:
class RussoAgent(Agent): # Agent is the superclass
    def __init__(self,  env:Env,
                # Additionnal parameters to be added here
                ):
        """
        Example agent implementation. Just picks a random action at each time step.
        """
        self.env = env
        self.count_remaining_weeks = 0
        self.default_action  = ACTION_NULL
        self.confinement_action = ACTION_CONFINE
        self.confinement_weeks_count = 0
        
    def load_model(self, savepath):
        # This is where one would define the routine for loading a pre-trained model
        pass

    def save_model(self, savepath):
        # This is where one would define the routine for saving the weights for a trained model
        pass

    def optimize_model(self):
        # This is where one would define the optimization step of an RL algorithm
        return 0
    
    def reset(self,env):
        # This should be called when the environment is reset (we do not loss any weight, no need to 
        # redefine actions, the environment is new and we need to save it)
        self.env = env
        self.count_remaining_weeks = 0
        self.confinement_weeks_count = 0
    
    def act(self, info):
        # this takes an observation and returns an action
        if self.count_remaining_weeks == 0:
            total_infected = info.total.infected # number of infected people at end of week 4
            
            if total_infected > 20000:
                self.count_remaining_weeks = 4
                self.confinement_weeks_count += 1
                return self.confinement_action
            
            else:
                return self.default_action
                
        else:
            self.count_remaining_weeks -= 1
            self.confinement_weeks_count += 1
            return self.confinement_action
            
agent = RussoAgent(env)

We now run a simulation applying Pr. Russo's Policy.

In [ ]:
""" Run the simulation """
log = []
finished = False
obs, info = env.reset(seed) # initialization (random infection)
agent.reset(env) # useless
agent.epsilon = 0 # taken from Agent, which is superclass
while not finished:
    action = agent.act(info)
    obs, R, finished, info = env.step(action)
    log.append(info) # save the information dict for logging
    if finished:
        break

total = {p:np.array([getattr(l.total,p) for l in log]) for p in dyn.parameters}
cities = {c:{p:np.array([getattr(l.city[c],p) for l in log]) for p in dyn.parameters} for c in dyn.cities}
actions = {a:np.array([l.action[a] for l in log]) for a in log[0].action.keys()}

#### Plots

In [ ]:
[plt.plot(y) for y in total.values()]
plt.legend(total.keys())
plt.ylabel('number of people in each state')
plt.xlabel('time (in weeks)')
plt.title('Unmitigated Epidemic')
plt.show()

In [ ]:
# andamento a plateau che susssegue la azione che prendiamo, ossia confinement
plt.plot(total['infected'], label = 'infected')
plt.plot(total['dead'], label = 'dead')
plt.legend()
plt.ylabel('number of people in each state')
plt.xlabel('time (in weeks)')
plt.title('Unmitigated Epidemic')
plt.show()

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(10,10))
row = col = 0
for idx, city in enumerate(cities.keys()):
    
    row = idx // 3
    col = idx % 3
    ax[row,col].plot(cities[city]['infected'],label = 'infected')
    ax[row,col].plot(cities[city]['dead'],label = 'dead')
    ax[row,col].set_title(city)
    
# choose title + common legend
plt.tight_layout()

In [ ]:
plt.imshow(np.array([v for v in actions.values()]).astype(np.uint8),aspect='auto')
plt.title('Actions')
plt.yticks([0,1,2,3], labels = list(actions.keys()))
plt.xlabel('time (in weeks)')
plt.show()

### Question 2.b

In order to be able to make meaningful conclusions, we implement the following evaluation procedure: we run 50 simulation episodes where actions are chosen from Pr. Russo's Policy.

Notice that, to make results reproducible, we initialize one seed for every episode in the simulation: the i-th simulation corresponds to `seed = i`

In [ ]:
agent = RussoAgent(env)
seeds = range(1,51)

# Initializing useful variables
conf_days = []
rewards = []
deaths = []

# Looping over 50 episodes
for trace in range(50): # for loop over episodes
    R_cumulative = 0
    finished = False
    obs, info = env.reset(seeds[trace]) # resetting the environment
    agent.reset(env)
    
    # Looping over 30 weeks
    for t in range(30):
        action = agent.act(info)
        obs, R, finished, info = env.step(action) 
        R_cumulative+= R.item()
        if finished:
            break
    """ Parse the logs """
    # Saving total number of confined days
    conf_days.append(7 * agent.confinement_weeks_count)
    # R_cumulative is computed in the inner loop
    rewards.append(R_cumulative)
    # Number of total deaths in the current episode
    deaths.append(info.total.dead)

#### Plots

In [ ]:
fig, ax = plt.subplots(3,figsize=(10,7))
def hist_avg(ax, data,title):
    ymax = 50
    if title == 'deaths':
        x_range = (1000,200000)
    elif title == 'cumulative rewards': 
        x_range = (-300,300)
    elif 'days' in title:
        x_range = (0,200)
    else:
        raise ValueError(f'{title} is not a valid title') 
    ax.set_title(title)
    ax.set_ylim(0,ymax)
    ax.vlines([np.mean(data)],0,ymax,color='red')
    ax.hist(data,bins=60,range=x_range)
hist_avg(ax[0], deaths,'deaths')
hist_avg(ax[1], rewards,'cumulative rewards')
hist_avg(ax[2], conf_days,'confined days')
fig.tight_layout()
plt.show()

""" Print example """
print(f'Average death number: {np.mean(deaths)}')
print(f'Average number of confined days: {np.mean(conf_days)}')
print(f'Average cumulative reward: {np.mean(rewards)}')

## Question 3

### Question 3.a

We start by defining action preprocessors and observation preprocessor to convert data from environment and neural network.

In [ ]:
# Action Preprocessor: the deafult action (Do Nothing) is encoded as 0, the CONFINEMENT action is encoded as 1

ACTION_NULL = 0
ACTION_CONFINE = 1

def action_preprocessor(a:torch.Tensor, dyn:ModelDynamics):
    action = { # DO NOTHING
        'confinement': False, 
        'isolation': False, 
        'hospital': False, 
        'vaccinate': False}
    
    if a == ACTION_CONFINE:
        action['confinement'] = True
        
    return action

# Observation Preprocessor: every observation is converted to a tensor containing the proportion of death and
# infected people in each city

SCALE = 1
def observation_preprocessor(obs: Observation, dyn:ModelDynamics):
    infected = SCALE * np.array([np.array(obs.city[c].infected)/obs.pop[c] for c in dyn.cities])
    dead = SCALE * np.array([np.array(obs.city[c].dead)/obs.pop[c] for c in dyn.cities])
    return torch.Tensor(np.stack((infected, dead))).unsqueeze(0)

We import useful packages, we initialize the environments making sure to define the appropriate observation and action spaces. This is essential, since the action and observation format needed by the dynamic model is different from the encoding taken as input by the neural network we are going to train.

In [ ]:
# Loading the environment that allows us to observe the duìynamics of the pandemic all over Switzerland
dyn = ModelDynamics('config/switzerland.yaml')   # load the switzerland map

# Initializing the environment
env = Env(  dyn, # We pass the dynamical model to the environment 
            action_space=spaces.Discrete(2) , # Here one could pass an openai gym action space that can then be sampled
            observation_space=spaces.Box(low=0, high=1, shape=(2,9,7)),
            action_preprocessor=action_preprocessor,
            observation_preprocessor=observation_preprocessor)

# If gpu is to be used
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

The following cell is standard and we do not have to adapt it to the problem we are solving. This is essential in order to create and define the **Replay Buffer**

In [ ]:
# The following is a named tuple representing a single transition in our environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

# The following is a cyclic buffer of bounded size that holds the transitions observed recently. 
# It also implements a .sample() method for selecting a random batch of transitions for training.
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

We now want to use a neural network to model the Q values for each (state, action) pair. The network architecture follows the suggestion provided in Table 4.

In [ ]:
class DQN(torch.nn.Module):

    def __init__(self, n_observations, n_actions):
        super().__init__()
        
        # MLP network
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(n_observations, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),  
            torch.nn.Linear(16, n_actions)
        )
        
    # Called in order to compute the forward pass in the network
    def forward(self, x):
        new_x = torch.flatten(x,1) ** (1/4) # since we might end up having very small input values due to the scaling
                                            # We flatten the input in order to use it in the linear network
        z = self.mlp(new_x)
        
        return z 

We now initiliaze the models we need and other additional useful variables that will be used in the training and evaluation procedure.

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
BATCH_SIZE = 2048
# GAMMA is the discount factor
GAMMA = 0.9
# EPS_START is the starting value of epsilon
EPS_START = 0.7
# EPS_END is the final value of epsilon (in case of decaying epsilon,meant to reduce the exploration)
EPS_MIN = 0.2 # same as EPS_START since we do not want to decrease it
# DECREASE_FLAG indicates whether we want to decrease the exploration
DECREASE_FLAG = False
# EXPLORATION_FLAG indicates whether we want to explore or not
EXPLORATION_FLAG = True
# LR is the learning rate of the AdamW optimizer
LR = 5e-3

# Get number of actions from gym action space
n_actions = env.action_space.n

# Get the number of state observations
obs, info = env.reset() # we should set the seed
n_observations = 2*9*7

policy_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to train
target_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to keep fixed for some iterates
target_net.load_state_dict(policy_net.state_dict())    # we initialize it as a copy of the policy net

optimizer = torch.optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True) # we initialize the optimizer
memory = ReplayMemory(20000) # we initialize the buffer from which we sample already observed transitions

We now initialize a function defining the epsilon greedy policy we use in order to observe transitions that will be later stored in the Replay Buffer.

In [ ]:
# The following function defines the policy deployed in order to explore and interact with the environments.
# In particular, it is an epsilon greedy policy based on the values provided by the network trained up to 
# this moment

def select_action(state, decrease_flag, exploration_flag, curr_episode):
    sample = random.random()

    num_episodes = 500
    
    # Setting the epsilon parameter to have or avoid exploration
    if exploration_flag:
        if decrease_flag:
            eps_threshold = max(EPS_START*(num_episodes-curr_episode)/num_episodes, EPS_MIN)
        else:
            eps_threshold = EPS_START
    else:
        eps_threshold = 0 # in this case we only consider the greedy policy determined by the network
        
    if sample > eps_threshold:
        with torch.no_grad():
            # We use torch.no_grad() since we are now using the network to simply take an action.
            # We do not need to expand the DAG.
            # We only consider the larger value to take the action.
            return policy_net(state).max(1)[1].view(1,1) # this format is needed to concatenate
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

The following function define the way the policy network is trained. The training happens as follow:
- BATCH_SIZE already observed trainsitions are sampled from the replay Buffer in order to train the newtork;

- In order to compute the loss (Huber Loss), we define the expected state action_values, which will be considered
  as a true label;
  
- We compute the loss and update the parameters using the optimizer (AdamW) initialized above.

In [ ]:
def optimize_model():
    
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch. This converts batch-array of Transitions to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements, moving the final results to the device
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    
    # We save the non final state in our sampling
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    
    # We save and concatenate the variables we need
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch)
    state_action_values = state_action_values.gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        # we compute max_a Q(s', a) in order to quantify the delta. We use the target net to improve stability
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = torch.nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
num_episodes = 110
num_eval_episodes = 20
num_weeks = 30
training_seeds = range(1, num_episodes + 1)
eval_seeds = range(num_episodes + 1, num_episodes + num_eval_episodes + 1)

training = []
training_weights = []
evaluation = []

makedirs('./checkpoints_task3', exist_ok=True)
rmtree('./checkpoints_task3')
makedirs('./checkpoints_task3')
PATH = './checkpoints_task3/policy_net'

for training_process in range(1, 2):
    
    print('\n TRAINING PROCESS :{} \n'.format(training_process))
    
    policy_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to train
    target_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to keep fixed for some iterates
    memory = ReplayMemory(20000) # we initialize the buffer from which we sample already observed transitions
    
    # Initialize list to keep trace of log rewards
    training_trace = []
    eval_trace = []

    for i_episode in range(num_episodes):
        
        if (i_episode % 10) == 0:
            print('Training episode :{}'.format(i_episode))

        # Initialize variable to keep trace of the total reward
        total_training_reward = 0
        # Initialize the environment and get its state (moving it to the device)
        state, info = env.reset(training_process*training_seeds[i_episode])
        state = torch.tensor(state, device=device)

        # We run an episode
        for t in range(num_weeks):

            action = select_action(state, DECREASE_FLAG, EXPLORATION_FLAG, i_episode)
            obs, reward, done, info = env.step(action.item())

            total_training_reward += reward.item()
            reward = torch.tensor([reward], device=device)

            if done: # in our case it corresponds to 30 weeks
                next_state = None
            else:
                next_state = torch.tensor(obs, device=device)

            # Store the transition in memory (in the replay buffer)
            memory.push(state, action, next_state, reward)

            # Move to the next state
            state = next_state

            if done:
                break

        # We log the cumulative reward to training trace
        training_trace.append(total_training_reward)

        # We run a training step on policy_net
        optimize_model()

        if ((i_episode + 1) % 5 == 0):

            # We update the target network every 5 epsiodes
            policy_net_state_dict = policy_net.state_dict()
            target_net.load_state_dict(policy_net_state_dict)

        if ((i_episode + 1) % 50 == 0) or ((i_episode + 1) == 500):

            print('Evaluation cycle: {}'.format(int((i_episode + 1) / 50)))
            average_rewards = []

            for new_episode in range(num_eval_episodes):

                total_eval_reward = 0
                state, info = env.reset(eval_seeds[new_episode])
                state = torch.tensor(state, device=device)

                for new_week in range(num_weeks):

                    action = select_action(state, DECREASE_FLAG, False, new_episode) # greedy policy when eploration_flag = False
                    obs, reward, done, info = env.step(action.item())
                    total_eval_reward += reward.item()

                    if done: # in our case it corresponds to 30 weeks
                        next_state = None
                    else:
                        next_state = torch.tensor(obs, device=device)

                    # Move to the next state
                    state = next_state

                    if done:
                        break

                average_rewards.append(total_eval_reward)

            eval_trace.append(np.mean(average_rewards))
            
    torch.save(policy_net.state_dict(), PATH + str(training_process) + '.pt')
    training.append(training_trace)
    training_weights.append(policy_net_state_dict)
    evaluation.append(eval_trace)

print('Complete')

In [ ]:
good_file = torch.load(PATH + str(training_process) + '.pt')
good_mod = DQN(n_observations, n_actions).to(device)
good_mod.load_state_dict(good_file)

print(good_mod.state_dict())

#### Plots

In [ ]:
print(evaluation[0])
e = [(evaluation[0][i] + evaluation[1][i] + evaluation[2][i])/3 for i in range(len(eval_trace))]

In [ ]:
plt.scatter(np.arange(len(training_trace)), training[0], label='training 1')
plt.scatter(np.arange(len(training_trace)), training[1], label='training 2')
plt.scatter(np.arange(len(training_trace)), training[2], label='training 3')
plt.scatter(np.arange(len(eval_trace)), e, label='evaluation')
plt.legend()
plt.ylabel('total reward trace')
plt.xlabel('episodes')

plt.title('Training and Evaluation Traces')
plt.show()

#### Question 3.b

In [ ]:
evaluation

Now we can implement a different exploration policy by adjusting some parameters. 

In particular, we want to implement a decrease in the exploration threshold.
This is done to advantage exploration over exploitation in the first stage of the training process, in order to speed up the learning process. The deeper we go in the training, the more information we have about the environment and the transitions, therefore we decrease the exploration coefficient in order to take advantage of exploitation. 

In [ ]:
# EPS_START is the starting value of epsilon
EPS_START = 0.7
# EPS_END is the final value of epsilon (in case of decaying epsilon, meant to reduce the exploration)
EPS_MIN = 0.2
# DECREASE_FLAG indicates whether we want to decrease the exploration
DECREASE_FLAG = True
# EXPLORATION_FLAG indicates whether we want to explore or not
EXPLORATION_FLAG = 1

In [ ]:
# The following function defines the policy deployed in order to explore and interact with the environments.
# In particular, it is an epsilon greedy policy based on the values provided by the network trained up to 
# this moment

def select_action(state, decrease_flag, exploration_flag, curr_episode):
    sample = random.random()
    
    # Setting the epsilon parameter to have or avoid exploration
    if exploration_flag:
        if decrease_flag:
            eps_threshold = max(EPS_START*(num_episodes-curr_episode)/num_episodes, EPS_MIN)
        else:
            eps_threshold = EPS_START
    else:
        eps_threshold = 0 # in this case we only consider the greedy policy determined by the network
        
    if sample > eps_threshold:
        with torch.no_grad():
            # We use torch.no_grad() since we are now using the network to simply take an action.
            # We do not need to expand the DAG.
            # We only consider the larger value to take the action.
            return policy_net(state).max(1)[1].view(1,1) # this format is needed to concatenate
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [ ]:
num_episodes = 50
num_eval_episodes = 20
num_weeks = 30
training_seeds = range(1,num_episodes + 1)
eval_seeds = range(num_episodes + 1, num_episodes + num_eval_episodes + 1)

# Initialize list to keep trace of log rewards
training_trace = []
eval_trace = []

for i_episode in range(num_episodes):
    
    print('Training episode :{}'.format(i_episode))
    # Initialize variable to keep trace of the total reward
    total_training_reward = 0
    # Initialize the environment and get its state (moving it to the device)
    state, info = env.reset(training_seeds[i_episode])
    state = torch.tensor(state, device=device)
    
    # We run an episode
    for t in range(num_weeks):
        
        action = select_action(state, DECREASE_FLAG, EXPLORATION_FLAG, i_episode)
        obs, reward, done, info = env.step(action.item())
        
        total_training_reward += reward.item()
        reward = torch.tensor([reward], device=device)
        
        if done: # in our case it corresponds to 30 weeks
            next_state = None
        else:
            next_state = torch.tensor(obs, device=device)

        # Store the transition in memory (in the replay buffer)
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        if done:
            break

    # We log the cumulative reward to training trace
    training_trace.append(total_training_reward)
            
    # We run a training step on policy_net
    optimize_model()
    
    if ((i_episode + 1) % 5 == 0):

        # We update the target network every 5 epsiodes
        policy_net_state_dict = policy_net.state_dict()
        target_net.load_state_dict(policy_net_state_dict)
    
    if ((i_episode + 1) % 50 == 0) or ((i_episode +1) == 500):
        
        average_rewards = []
        
        for new_episode in range(num_eval_episodes):
            
            total_eval_reward = 0
            state, info = env.reset(eval_seeds[new_episode])
            state = torch.tensor(state, device=device)
            
            for new_week in range(num_weeks):
                
                action = select_action(state, DECREASE_FLAG, 0, new_episode) # greedy policy when eploration_flag = 0
                obs, reward, done, info = env.step(action.item())
                total_eval_reward += reward.item()
                
                if done: # in our case it corresponds to 30 weeks
                    next_state = None
                else:
                    next_state = torch.tensor(obs, device=device)
                
                # Move to the next state
                state = next_state
                
                if done:
                    break
            
            average_rewards.append(total_eval_reward)
        
        eval_trace.append(np.mean(average_rewards))

print('Complete')

In [ ]:
plt.scatter(np.arange(len(training_trace)), training_trace, label='training')
plt.scatter(np.arange(len(eval_trace)), eval_trace, label='training')
plt.legend()
plt.ylabel('total reward trace')
plt.xlabel('episodes')
plt.title('Training and Evaluation Traces')
plt.show()

## Question 4.1

Setting the parameters

In [ ]:
DECRESE_FLAG = True
LR = 10e-5

### Question 4.a

con 5 neuroni in più nell'input (embedding dell'azione alla settimana prima) ci fa risparmiare 2^5 neuroni in uscita
in uscita abbiamo quindi 5 neuroni invece che 2^5

nel caso tabular con (stato agnostico alla situazione attuale sulle azioni), in output dovremmo quindi avere 2^5 neuroni, il che vorrebbe dire che potenzialmente potremmo prendere più 'toggle' contemporaneamente, che non è possibile

dando informazioni sull'azione in input, invece, possiamo ridurre l'output a 5 neuroni e poi applicare una softmax per scegliere quale toggle applicare
In questo modo imponiamo che possa essere switchata solo una azione alla volta

In [ ]:
# Action Preprocessor: the deafult action (Do Nothing) is encoded as 0, the CONFINEMENT action is encoded as 1

TOGGLE_NULL = 0
TOGGLE_CONFINEMENT = 1
TOGGLE_ISOLATION = 2
TOGGLE_HOSPITAL = 3
TOGGLE_VACCINATION = 4

def action_preprocessor(a:torch.Tensor, dyn:ModelDynamics):
    
    default_action = { # DO NOTHING
        'confinement': False, 
        'isolation': False, 
        'hospital': False, 
        'vaccinate': False}
    
    if a not in range(0, 5):
        print('Not a valid action!')
        return default_action
    
    
    action = dyn.get_action()
    
    if a == TOGGLE_NULL: 
        return action
    elif a == TOGGLE_CONFINEMENT:
        action['confinement'] = not action['confinement'] 
    elif a == TOGGLE_ISOLATION:
        action['isolation'] = not action['isolation']
    elif a == TOGGLE_HOSPITAL:
        action['hospital'] = not action['hospital']
    else: 
        action['vaccinate'] = not action['vaccinate']
    
    return action

# Observation Preprocessor: every observation is converted to a tensor containing the proportion of death and
# infected people in each city

SCALE = 1
def observation_preprocessor(obs: Observation, dyn:ModelDynamics):
    infected = SCALE * np.array([np.array(obs.city[c].infected)/obs.pop[c] for c in dyn.cities])
    dead = SCALE * np.array([np.array(obs.city[c].dead)/obs.pop[c] for c in dyn.cities])
    
    curr_action = torch.Tensor(4)
    curr_action[0] = 1 if dyn.get_action()['confinement'] else 0
    curr_action[1] = 1 if dyn.get_action()['isolation'] else 0
    curr_action[2] = 1 if dyn.get_action()['hospital'] else 0
    curr_action[3] = 1 if dyn.get_action()['vaccinate'] else 0
    
    return (torch.Tensor(np.stack((infected, dead))).unsqueeze(0), curr_action.unsqueeze(dim=0))

In [ ]:
# Loading the environment that allows us to observe the duìynamics of the pandemic all over Switzerland
dyn = ModelDynamics('config/switzerland.yaml')   # load the switzerland map

# Initializing the environment
env = Env(  dyn, # We pass the dynamical model to the environment 
            action_space=spaces.Discrete(5) , # Here one could pass an openai gym action space that can then be sampled
            # observation_space=spaces.Box(low=0, high=1, shape=(2,9,7)),
            action_preprocessor=action_preprocessor,
            observation_preprocessor=observation_preprocessor)

# If gpu is to be used
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
# The following is a named tuple representing a single transition in our environment
Transition = namedtuple('Transition',
                        ('state', 'encoded_current_action', 'action', 'next_state','next_encoded_action', 'reward'))

# The following is a cyclic buffer of bounded size that holds the transitions observed recently. 
# It also implements a .sample() method for selecting a random batch of transitions for training.
class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(torch.nn.Module):

    def __init__(self, n_observations, n_actions):
        super().__init__()
        
        # MLP network
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(n_observations + 4, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),  
            torch.nn.Linear(16, n_actions)
        )
        
    # Called in order to compute the forward pass in the network
    def forward(self, x):
        new_x = torch.flatten(x[0],1) ** (1/4) # since we might end up having very small input values due to the scaling
                                            # We flatten the input in order to use it in the linear network
        
        new_y = torch.flatten(x[1],1)
        
        z = torch.cat((new_x, new_y), dim=1)
        
        return self.mlp(z)

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
BATCH_SIZE = 10
# GAMMA is the discount factor
GAMMA = 0.9
# EPS_START is the starting value of epsilon
EPS_START = 0.7
# EPS_END is the final value of epsilon (in case of decaying epsilon,meant to reduce the exploration)
EPS_MIN = 0.2 # same as EPS_START since we do not want to decrease it
# DECREASE_FLAG indicates whether we want to decrease the exploration
DECREASE_FLAG = False
# EXPLORATION_FLAG indicates whether we want to explore or not
EXPLORATION_FLAG = True
# LR is the learning rate of the AdamW optimizer
LR = 5e-3

# Get number of actions from gym action space
n_actions = env.action_space.n

# Get the number of state observations
obs, info = env.reset() # we should set the seed
n_observations = 2*9*7

policy_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to train
target_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to keep fixed for some iterates
target_net.load_state_dict(policy_net.state_dict())    # we initialize it as a copy of the policy net

optimizer = torch.optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True) # we initialize the optimizer
memory = ReplayMemory(20000) # we initialize the buffer from which we sample already observed transitions

In [ ]:
# The following function defines the policy deployed in order to explore and interact with the environments.
# In particular, it is an epsilon greedy policy based on the values provided by the network trained up to 
# this moment

def select_action(state, decrease_flag, exploration_flag, curr_episode):
    sample = random.random()
    
    # Setting the epsilon parameter to have or avoid exploration
    if exploration_flag:
        if decrease_flag:
            eps_threshold = max(EPS_START*(num_episodes-curr_episode)/num_episodes, EPS_MIN)
        else:
            eps_threshold = EPS_START
    else:
        eps_threshold = 0 # in this case we only consider the greedy policy determined by the network
        
    if sample > eps_threshold:
        with torch.no_grad():
            # We use torch.no_grad() since we are now using the network to simply take an action.
            # We do not need to expand the DAG.
            # We only consider the larger value to take the action.
            return policy_net(state).max(1)[1].view(1,1) # this format is needed to concatenate
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)

In [ ]:
def optimize_model():
    
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch. This converts batch-array of Transitions to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements, moving the final results to the device
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool)
    
    # We save the non final state in our sampling
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    non_final_next_actions = torch.cat([a for a in batch.next_encoded_action if a is not None])
    
    
    # We save and concatenate the variables we need
    state_batch = torch.cat(batch.state)
    encoded_current_action_batch = torch.cat(batch.encoded_current_action)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)
    
    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net((state_batch, encoded_current_action_batch))
    state_action_values = state_action_values.gather(1, action_batch)
    
    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        # we compute max_a Q(s', a) in order to quantify the delta. We use the target net to improve stability
        next_state_values[non_final_mask] = target_net((non_final_next_states, non_final_next_actions)).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = torch.nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
num_episodes = 110
num_eval_episodes = 20
num_weeks = 30
training_seeds = range(1, num_episodes + 1)
eval_seeds = range(num_episodes + 1, num_episodes + num_eval_episodes + 1)

training = []
training_weights = []
evaluation = []

makedirs('./checkpoints_task4', exist_ok=True)
rmtree('./checkpoints_task4')
makedirs('./checkpoints_task4')
PATH = './checkpoints_task4/policy_net'

for training_process in range(1, 2):
    
    print('\n TRAINING PROCESS :{} \n'.format(training_process))
    
    policy_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to train
    target_net = DQN(n_observations, n_actions).to(device) # this is the network we are going to keep fixed for some iterates
    memory = ReplayMemory(20000) # we initialize the buffer from which we sample already observed transitions
    
    # Initialize list to keep trace of log rewards
    training_trace = []
    eval_trace = []

    for i_episode in range(num_episodes):
        
        if (i_episode % 10) == 0:
            print('Training episode :{}'.format(i_episode))

        # Initialize variable to keep trace of the total reward
        total_training_reward = 0
        # Initialize the environment and get its state (moving it to the device)
        state, info = env.reset(training_process*training_seeds[i_episode])
        
        curr_state = torch.tensor(state[0], device=device)
        encoded_action = torch.tensor(state[1], device = device)

        # We run an episode
        for t in range(num_weeks):

            action = select_action((curr_state, encoded_action), DECREASE_FLAG, EXPLORATION_FLAG, i_episode)
            obs, reward, done, info = env.step(action.item())
            
            total_training_reward += reward.item()
            reward = torch.tensor([reward], device=device)

            if done: # in our case it corresponds to 30 weeks
                next_state = None
                next_encoded_action= None
            else:
                next_state = torch.tensor(obs[0], device=device)
                next_encoded_action = torch.tensor(obs[1], device=device)

            # Store the transition in memory (in the replay buffer)
            memory.push(curr_state, encoded_action, action, next_state, next_encoded_action, reward)

            # Move to the next state
            curr_state = next_state
            encoded_action = next_encoded_action

            if done:
                break

        # We log the cumulative reward to training trace
        training_trace.append(total_training_reward)

        # We run a training step on policy_net
        optimize_model()

        if ((i_episode + 1) % 5 == 0):

            # We update the target network every 5 epsiodes
            policy_net_state_dict = policy_net.state_dict()
            target_net.load_state_dict(policy_net_state_dict)

        if ((i_episode + 1) % 50 == 0) or ((i_episode + 1) == 500):

            print('Evaluation cycle: {}'.format(int((i_episode + 1) / 50)))
            average_rewards = []

            for new_episode in range(num_eval_episodes):

                total_eval_reward = 0
                state, info = env.reset(eval_seeds[new_episode])
                
                curr_state = torch.tensor(state[0], device=device)
                encoded_action = torch.tensor(state[1], device = device)

                for new_week in range(num_weeks):

                    action = select_action((curr_state, encoded_action), DECREASE_FLAG, False, new_episode)
                    obs, reward, done, info = env.step(action.item())

                    total_training_reward += reward.item()
                    reward = torch.tensor([reward], device=device)

                    if done: # in our case it corresponds to 30 weeks
                        next_state = None
                        next_encoded_action= None
                    else:
                        next_state = torch.tensor(obs[0], device=device)
                        next_encoded_action = torch.tensor(obs[1], device=device)

                    # Move to the next state
                    curr_state = next_state
                    encoded_action = next_encoded_action
                    
                    if done:
                        break

                average_rewards.append(total_eval_reward)

            eval_trace.append(np.mean(average_rewards))
            
    torch.save(policy_net.state_dict(), PATH + str(training_process) + '.pt')
    training.append(training_trace)
    training_weights.append(policy_net_state_dict)
    evaluation.append(eval_trace)

print('Complete')

#### Question 4.1.d

la distanza (in termini di metrica) fra un'azione e la successiva deve essere al più 1
ossia possiamo fare il toggling solo di una azione alla volta (al massimo... volendo si può anche non cambiare nulla)

In [ ]:
good_file = torch.load(PATH + str(training_process) + '.pt')
good_mod = DQN(n_observations, n_actions).to(device)
good_mod.load_state_dict(good_file)

print(good_mod.state_dict())